In [1]:
# Python program to create
# Image Classifier using CNN

# Importing the required libraries
import tensorflow as tf
from tflearn import regression
from tflearn import input_data, dropout, fully_connected
from tflearn import conv_2d, max_pool_2d
from flask import Flask
import tflearn
import glob
import cv2
import os
import numpy as np
from random import shuffle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

'''Setting up the env'''

TRAIN_DIR = 'train/'
TEST_DIR = 'test/'
UPLOADS_DIR = 'uploads/'
IMG_SIZE = 100
LR = 1e-3


'''Setting up the model which will help with tensorflow models'''
# MODEL_NAME = 'downvswithout-{}-{}.model'.format(LR, '6conv-basic')
classes = ['22q11', 'Down', 'Angelman', 'Apert', 'CDL', 'FragileX', 'Marfan', 'Progeria', 'Sotos', 'TreacherCollins', 'Turner', 'Williams', 'Without']
labels = [0]*len(classes)
model = None

def label_img(cond): 
    ele = classes.index(cond)
    ret = labels.copy();
    ret[ele] = 1
    return ret

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
def create_train_data():
    training_data = []
    for cond in classes:
        for img in tqdm(os.listdir(TRAIN_DIR+'/'+cond)):
            # labeling the images
            label = label_img(cond)

            path = os.path.join(TRAIN_DIR+'/'+cond, img)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

            try:
#                 img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
#                 faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
#                 faces = faceCascade.detectMultiScale(img)
#                 for (x, y, w, h) in faces:
#                     img = img[y:y+h, x:x+w]
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            except Exception as ex:
                continue
            
            # final step-forming the training data list with numpy array of the images
            training_data.append([np.array(img), np.array(label), cond])

    shuffle(training_data)

    # saving our trained data for further uses if required
    np.save('train_data.npy', training_data)
    return training_data


In [3]:
def process_test_data():
    testing_data = []
    classes = ['Without']
    for cond in classes:
        for img in tqdm(os.listdir(TEST_DIR+'/'+cond)):
            path = os.path.join(TEST_DIR+'/'+cond, img)
            label = cond
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            try:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            except:
                continue
            testing_data.append([np.array(img), label, cond])
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data


In [4]:
def training_model():
    global model
    
    tf.reset_default_graph()
    convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

    convnet = conv_2d(convnet, 32, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 64, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 128, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 64, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = conv_2d(convnet, 32, 5, activation='relu')
    convnet = max_pool_2d(convnet, 5)

    convnet = fully_connected(convnet, 1024, activation='relu')
    convnet = dropout(convnet, 0.8)

    convnet = fully_connected(convnet, 13, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=LR,
      loss='categorical_crossentropy', name='targets')

    model = tflearn.DNN(convnet, tensorboard_dir='log')
    
    if os.path.exists("./model/model.tfl.index"):
        print("Model exists")
        model.load("model/model.tfl")
    else:
        train_data = create_train_data()
        length = len(train_data)//2
    
        train = train_data[:-length]
        test = train_data[-length:]

        '''Setting up the features and lables'''
        # X-Features & Y-Labels

        X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        Y = [i[1] for i in train]
        test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        test_y = [i[1] for i in test]

        '''Fitting the data into our model'''
        model.fit({'input': X}, {'targets': Y}, n_epoch = 2, 
        validation_set =({'input': test_x}, {'targets': test_y}), 
        snapshot_step = 500, show_metric = True, run_id="disease_detection") 
        model.save("model/model.tfl")   
        
#     Splitting the testing data and training data
    

In [5]:
def testing_model(path):
    output = []
    global model
    try:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = faceCascade.detectMultiScale(
            img,
            scaleFactor=1.3,
            minNeighbors=3,
            minSize=(30, 30)
        )
        for (x, y, w, h) in faces:
            img = img[y:y+h, x:x+w]
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        data = img.reshape(IMG_SIZE, IMG_SIZE, 1) 
        model_out = model.predict([data])[0]
        return ["output", model_out, classes[np.argmax(model_out)]]
    except Exception as ex:
        return str(ex)

In [6]:
# training_model()

In [7]:
# testing_model('uploads/jatin.jpg')